In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import fitz  
from PIL import Image
import io
import base64
import mimetypes

TOGETHER_API_KEY = "<Your API Key>"
MODEL_NAME = "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8"

def load_file(file_path):
    mime_type, _ = mimetypes.guess_type(file_path)
    if file_path.endswith(".csv"):
        return pd.read_csv(file_path)
    elif file_path.endswith(".xlsx") or file_path.endswith(".xls"):
        return pd.read_excel(file_path)
    elif file_path.endswith(".txt"):
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()
    elif file_path.endswith(".pdf"):
        with fitz.open(file_path) as doc:
            text = ""
            for page in doc:
                text += page.get_text()
            return text
    elif mime_type and mime_type.startswith("image"):
        img = Image.open(file_path)
        return img
    else:
        return "Unsupported file format."

def ask_llama(prompt):
    headers = {
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "max_tokens": 512,
        "temperature": 0.7,
        "top_p": 0.9,
        "repetition_penalty": 1.1
    }

    response = requests.post("https://api.together.xyz/v1/completions", headers=headers, json=data)

    if response.status_code == 200:
        return response.json()["choices"][0]["text"].strip()
    else:
        return f"Error: {response.status_code}, {response.text}"

def plot_histogram(df):
    numeric_cols = df.select_dtypes(include='number').columns
    if len(numeric_cols) > 0:
        df[numeric_cols[0]].hist(bins=20)
        plt.title(f"Histogram of {numeric_cols[0]}")
        plt.xlabel(numeric_cols[0])
        plt.ylabel("Frequency")
        plt.show()

def plot_correlation(df):
    numeric_cols = df.select_dtypes(include='number')
    if not numeric_cols.empty:
        corr = numeric_cols.corr()
        sns.heatmap(corr, annot=True, cmap='coolwarm')
        plt.title("Correlation Heatmap")
        plt.show()

file_path = "uploaded_files/sample.csv"  
data = load_file(file_path)

if isinstance(data, pd.DataFrame):
    print("✅ Loaded CSV/Excel file as DataFrame")
    print(data.head())
    
    prompt = f"""This is the data:\n{data.head().to_string()}\nGive me a summary analysis."""
    response = ask_llama(prompt)
    print("\n📈 LLaMA Summary:\n", response)

    plot_histogram(data)
    plot_correlation(data)

elif isinstance(data, str):
    print("✅ Loaded text/PDF")
    print(data[:300])
    
    prompt = f"""Analyze this document:\n{data[:2000]}"""
    response = ask_llama(prompt)
    print("\n📄 LLaMA Summary:\n", response)

elif isinstance(data, Image.Image):
    print("✅ Loaded image file")
    display(data)

    response = ask_llama("This is an image. Try to describe what kind of data it might contain.")
    print("\n🖼️ LLaMA Image Response:\n", response)

else:
    print("⚠️ Unsupported file type or failed to load.")
